1.1 UserCF

In [2]:
import os
import numpy as np
import pandas as pd
from surprise import Reader
from surprise import accuracy
from surprise import Dataset
from surprise.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from collections import defaultdict

In [3]:
from surprise import SVD,KNNBasic,KNNWithMeans,KNNWithZScore,SVDpp,AlgoBase,BaselineOnly

    使用ml-latest-small数据集

In [4]:
#moviesPath = 'datasets\\ml-latest-small\\movies.csv'
ratingsPath = 'datasets\\ml-latest-small\\ratings.csv'
#moviesDF = pd.read_csv(moviesPath,index_col=None)
ratingsDF = pd.read_csv(ratingsPath,index_col=None).loc[:,['userId','movieId','rating']]

# file_path = os.path.expanduser("D:\\DevelopSoft\\pycharm\\Projects\\Andrew_Ng\\Surprise\\firstPaper\\datasets\\ml-latest-small\\ratings.csv")
# reader = Reader(line_format='user item rating timestamp',sep=',')
# data = Dataset.load_from_file(file_path,reader=reader)

In [5]:
# 尝试一把使用Baselineonly()
# cross_validate(BaselineOnly(),data,verbose=True)

In [6]:
# 尝试一把使用SVD
# algo = SVD()
# cross_validate(algo,data,measures=['RMSE','MAE'],cv=5,verbose=True)

    数据集切分

In [7]:
ratingsDF.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [8]:
# 按照6:2:2的比例将数据集进行拆分，同时打印出总的用户和电影数量，
# 训练集中的用户和电影数量，交叉验证集中的用户和电影数量，以及测试集中的用户和电影数量：
trainRatDF,testRatingsDF = train_test_split(ratingsDF,test_size=0.2)
trainRatingsDF,crossValidationRatingsDF = train_test_split(trainRatDF,test_size=0.25)

print("total_movie_count:"+str(len(set(ratingsDF['movieId'].values.tolist()))))
print("total_user_count:"+str(len(set(ratingsDF['userId'].values.tolist()))))
print("train_movie_count:"+str(len(set(trainRatingsDF['movieId'].values.tolist()))))
print("train_user_count:"+str(len(set(trainRatingsDF['userId'].values.tolist()))))
print("crossValidation_movie_count:"+str(len(set(crossValidationRatingsDF['movieId'].values.tolist()))))
print("crossValidation_user_count:"+str(len(set(crossValidationRatingsDF['userId'].values.tolist()))))
print("test_movie_count:"+str(len(set(testRatingsDF['movieId'].values.tolist()))))
print("test_user_count:"+str(len(set(testRatingsDF['userId'].values.tolist()))))

total_movie_count:9724
total_user_count:610
train_movie_count:8133
train_user_count:610
crossValidation_movie_count:5073
crossValidation_user_count:609
test_movie_count:5134
test_user_count:609


In [9]:
print(len(ratingsDF))
print(len(trainRatingsDF))
print(len(crossValidationRatingsDF))
print(len(testRatingsDF))

100836
60501
20167
20168


In [10]:
trainRatingsDF.describe()

,userId,movieId,rating
count,60501.000000,60501.000000,60501.000000
mean,325.461249,19485.160427,3.496248
std,182.818644,35576.582652,1.044728
min,1.000000,1.000000,0.500000
25%,177.000000,1198.000000,3.000000
50%,325.000000,2987.000000,3.500000
75%,477.000000,8132.000000,4.000000
max,610.000000,193585.000000,5.000000


In [11]:
reader = Reader(rating_scale=(0.5,5))
traindata = Dataset.load_from_df(trainRatingsDF,reader)

In [12]:
trainset = traindata.build_full_trainset()
# algo = KNNBasic()
# algo.fit(trainset)

In [13]:
# from surprise.model_selection import KFold

# kf = KFold(n_splits=3)

# algo = SVD()

In [14]:
# for trainset,testset in kf.split(traindata):
#     # train and test algorithm.
#     algo.fit(trainset)
#     predictions = algo.test(testset)
#     #print(predictions)
    
#     # compute and print Root Mean Squared Error
#     accuracy.rmse(predictions,verbose=True)

1.2 基于内容的推荐算法

In [15]:
# 此处仅用电影的题材类别表示电影内容
moviesPath = 'datasets\\ml-latest-small\\movies.csv'

moviesDF = pd.read_csv(moviesPath,index_col=None).loc[:,['movieId','genres']]

In [16]:
moviesDF.head()

,movieId,genres
0,1,Adventure|Animation|Children|Comedy|Fantasy
1,2,Adventure|Children|Fantasy
2,3,Comedy|Romance
3,4,Comedy|Drama|Romance
4,5,Comedy


In [17]:
genre_list = ['Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','IMAX','(no genres listed)']

In [18]:
genre_int_map = {val: ii for ii, val in enumerate(genre_list)}
# print(len(genre_int_map))
# genre_int_map

In [19]:
moviesDF.loc[0,'genres']

'Adventure|Animation|Children|Comedy|Fantasy'

In [20]:
# 电影题材可以视为多值属性，且类型不多，可以直接使用Multi-Hot编码
def genres_multi_hot(genre_int_map):
    """
    电影题材类型使用multi-hot编码
    :param: genre_int_map:genre到数字的映射字典
    :return:
    """
    def helper(genres):
        genre_int_list = [genre_int_map[genre] for genre in genres.split('|')]
        multi_hot = np.zeros(len(genre_int_map))
        multi_hot[genre_int_list] = 1
        return multi_hot

    return helper

moviesDF['genresMultiHot'] = moviesDF['genres'].map(genres_multi_hot(genre_int_map))

In [21]:
# 也可以不写函数，而是使用for循环遍历moviesDF
'''
moviesDF['genres']=moviesDF['genres'].str.split('|')
genre_list = ['Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','IMAX','(no genres listed)']
genre_int_map = {val: ii for ii, val in enumerate(genre_list)}
print(len(genre_int_map))
genre_int_map
# 给moviesDF添加一列
moviesDF['genresMultiHot'] = None
for i in range(len(moviesDF)):
    genre_int_list = [genre_int_map[genre] for genre in moviesDF.loc[i,'genres']]
    multi_hot = np.zeros(len(genre_int_map))
    multi_hot[genre_int_list] = 1
    moviesDF.at[i,'genresMultiHot'] = multi_hot

moviesDF['genresMultiHot'][122]
'''

"\nmoviesDF['genres']=moviesDF['genres'].str.split('|')\ngenre_list = ['Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western','IMAX','(no genres listed)']\ngenre_int_map = {val: ii for ii, val in enumerate(genre_list)}\nprint(len(genre_int_map))\ngenre_int_map\n# 给moviesDF添加一列\nmoviesDF['genresMultiHot'] = None\nfor i in range(len(moviesDF)):\n    genre_int_list = [genre_int_map[genre] for genre in moviesDF.loc[i,'genres']]\n    multi_hot = np.zeros(len(genre_int_map))\n    multi_hot[genre_int_list] = 1\n    moviesDF.at[i,'genresMultiHot'] = multi_hot\n\nmoviesDF['genresMultiHot'][122]\n"

In [22]:
moviesDF['genresMultiHot'][122]

array([0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0.])

In [23]:
len(moviesDF)

9742

In [24]:
# 计算电影内容的相似度矩阵并存入文件
# 这里我们使用余弦相似度来计算电影之间的相似度关系
def calCosineSimilarity(list1,list2):
    res = 0
    denominator1 = 0 # denominator分母
    denominator2 = 0
    for (val1,val2) in zip(list1,list2):
        res += (val1 * val2)
        denominator1 += val1 ** 2
        denominator2 += val2 ** 2
    return res / (np.sqrt(denominator1 * denominator2))   
# 计算电影之间的相似度矩阵，对于用户相似度矩阵，这是一个对称矩阵，同时对角线的元素为1，所以我们只需要计算上三角矩阵的值即可
movieSimMatrix = np.ones((len(moviesDF),len(moviesDF)),dtype=np.float32)
for i in range(len(moviesDF)-1):
    for j in range(i+1,len(moviesDF)):
        movieSimMatrix[i,j] = calCosineSimilarity(moviesDF['genresMultiHot'][i],moviesDF['genresMultiHot'][j])
        movieSimMatrix[j,i] = movieSimMatrix[i,j]

In [25]:
row = np.arange(len(moviesDF))
row

array([   0,    1,    2, ..., 9739, 9740, 9741])

In [26]:
movieSimMatrix = np.insert(movieSimMatrix,0,values=row,axis=0)
movieSimMatrix[0]

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 9.739e+03, 9.740e+03,
       9.741e+03], dtype=float32)

In [27]:
movieSimMatrix = np.delete(movieSimMatrix,0,axis=0)
movieSimMatrix.shape

(9742, 9742)

In [28]:
movieSimMatrix

array([[1.        , 0.7745967 , 0.31622776, ..., 0.        , 0.31622776,
        0.4472136 ],
       [0.7745967 , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31622776, 0.        , 1.        , ..., 0.        , 0.        ,
        0.70710677],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.31622776, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.4472136 , 0.        , 0.70710677, ..., 0.        , 0.        ,
        1.        ]], dtype=float32)

In [29]:
col = np.arange(len(moviesDF))
col

array([   0,    1,    2, ..., 9739, 9740, 9741])

In [30]:
col = np.concatenate(([-1],col))
col.shape

(9743,)

In [31]:
movieSimMatrix = np.insert(movieSimMatrix,0,values=col,axis=1)

ValueError: could not broadcast input array from shape (9743,1) into shape (9742,1)

In [514]:
np.savetxt('datasets\\ml-latest-small\\movieSim.csv',movieSimMatrix,delimiter = ',')

In [518]:
movieSimMatrix[9742]

array([9.741000e+03, 4.472136e-01, 0.000000e+00, ..., 0.000000e+00,
       0.000000e+00, 1.000000e+00], dtype=float32)

In [485]:
# a = np.zeros((1,1),dtype=np.float64)
# import sys
# print(a)
# print(sys.getsizeof(a))
# a.itemsize

[[0.]]
120


8

In [470]:
# import sys
# sys.getsizeof(movieSimMatrix[0][0])


26

In [475]:
# movieSimMatrix[0][0].nbytes

2

In [32]:
from six import get_unbound_function as guf

class MyAlgoBase(AlgoBase):
    
    def __init__(self, **kwargs):
        AlgoBase.__init__(self, **kwargs)
        
    def fit(self, trainset):
        
        if (guf(self.__class__.train) is not guf(AlgoBase.train) and
                not self.skip_train):
            self.train(trainset)
            return 
        self.skip_train = False
        
        self.trainset = trainset
        
        self.bu = self.bi = None
        
        return self
    
    def compute_similarities(self, verbose=False):
        """Build the similarity matrix.
        
        The way the similarity matrix is computed depends on the 
        ''sim_options'' parameter passed at the creation of the algorithm (see
        :ref:'similarity_measures_configuration')
        
        This methdod is only relevant for algorithms using a similarity measure,
        such as the :ref:'k-NN algorithms <pred_package_knn_ispired>'.
        
        Args:
            verbose(bool) : if ''True'',print status message. Default is ''False''.
            
        Returns:
            The similarity matrix."""
            
#         complex_sim_mat = pd.read_csv('D:\\Spyder\\surprise\\paper_code\\ml-latest-small-data\\LY_data\\complex_sim_df_82.csv')
        complex_sim_mat = pd.read_csv('datasets\\ml-latest-small\\movieSim.csv')
        complex_sim_mat = complex_sim_mat.set_index(['Unnamed: 0'])
        complex_sim_mat.columns = list(complex_sim_mat.index)
        
        construction_func = {'cosine': sims.cosine,
                             'msd':sims.msd,
                             'pearson':sims.pearson,
                             'pearson_baseline':sims.pearson_baseline}
        
        
        if self.sim_options['user_based']:
            n_x, yr = self.trainset.n_users, self.trainset.ir
        else:
            n_x, yr = self.trainset.n_items, self.trainset.ur
            
        min_support = self.sim_options.get('min_support', 1)
        
        args = [n_x, yr, min_support]
        
        
        
        name = self.sim_options.get('name', 'msd').lower()   
        
        
        if name == 'pearson_baselines':
            shrinkage = self.sim_options.get('shrinkage', 100)
            bu, bi = self.compute_baselines()
            if self.sim_options['user_based']:
                bx, by = bu, bi
            else:
                bx, by = bi, bu
                
            args += [self.trainset.global_mean, bx, by, shrinkage]
        
            
        try:
            if verbose:
                print('Computing the {0} similarity matrix...'.format(name))
            sim = complex_sim_mat
            if verbose:
                print('Done computing similarity matrix.')
            return sim
        except KeyError:
            raise NameError('Wrong sim name ' + name + '.Allowed values ' +
                            'are ' + ', '.join(construction_func.keys()) + '.')


In [33]:
class MySymmetricAlgo(MyAlgoBase):
    
    def __init__(self, sim_options={}, verbose=True, **kwargs):
        MyAlgoBase.__init__(self, sim_options=sim_options, **kwargs)
        self.verbose = verbose
        
    def fit(self, trainset):
        MyAlgoBase.fit(self, trainset)
        
        ub = self.sim_options['user_based']
        self.n_x = self.trainset.n_users if ub else self.trainset.n_items
        self.n_y = self.trainset.n_items if ub else self.trainset.n_users
        self.xr = self.trainset.ur if ub else self.trainset.ir
        self.yr = self.trainset.ir if ub else self.trainset.ur
        
        return self
        
    def switch(self, u_stuff, i_stuff):
        
        if self.sim_options['user_based']:
            return u_stuff, i_stuff
        else:
            return i_stuff, u_stuff

In [34]:
class MyKnnWithMeans(MySymmetricAlgo):
    
    def __init__(self, k=40, min_k=1, sim_options={}, verbose=True, **kwargs):

        MySymmetricAlgo.__init__(self, sim_options=sim_options,
                               verbose=verbose, **kwargs)

        self.k = k
        self.min_k = min_k
        
    def fit(self, trainset):

        MySymmetricAlgo.fit(self, trainset)
        self.sim = self.compute_similarities(verbose=self.verbose)

        self.means = np.zeros(self.n_x)
        for x, ratings in iteritems(self.xr):
            self.means[x] = np.mean([r for (_, r) in ratings])

        return self
        
    def estimate(self, u, i):

        if not (self.trainset.knows_user(u) and self.trainset.knows_item(i)):
            raise PredictionImpossible('User and/or item is unkown.')

        x, y = self.switch(u, i)


        #neighbors = [(x2, self.sim[x, x2], r) for (x2, r) in self.yr[y]]
        #k_neighbors = heapq.nlargest(self.k, neighbors, key=lambda t: t[1])
        
        neighbors = [(x2, self.sim.loc[x, x2], r) for (x2, r) in self.yr[y]]
        k_neighbors = heapq.nlargest(self.k, neighbors, key=lambda t: t[1])

        est = self.means[x]

        # compute weighted average
        sum_sim = sum_ratings = actual_k = 0
        for (nb, sim, r) in k_neighbors:
            if sim > 0:
                sum_sim += sim
                sum_ratings += sim * (r - self.means[nb])
                actual_k += 1

        if actual_k < self.min_k:
            sum_ratings = 0

        try:
            est += sum_ratings / sum_sim
        except ZeroDivisionError:
            pass  # return mean

        details = {'actual_k': actual_k}
        return est, details

In [35]:
sim_options = {'name': 'pearson',
               'user_based': False  # compute  similarities between items
               }

In [36]:
algo = MyKnnWithMeans(sim_options=sim_options)

In [37]:
algo.fit(trainset)

ValueError: Length mismatch: Expected axis has 9742 elements, new values have 9743 elements

In [ ]:
crossValidationRatingsdata = Dataset.load_from_df(crossValidationRatingsDF,reader)
crossValidationset = crossValidationRatingsdata.build_full_trainset().build_testset()
predictions = algo.test(crossValidationset)

In [ ]:
crossValidationRatingsDF.head()

In [ ]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [ ]:
precisions, recalls = precision_recall_at_k(predictions, threshold=3.5)

In [ ]:
len(predictions)

In [ ]:
avg_prec = sum(prec for prec in precisions.values()) / len(precisions)
avg_recall = sum(rec for rec in recalls.values()) / len(recalls)
avg_prec = round(avg_prec, 4)
avg_recall = round(avg_recall, 4)
rmse = round(accuracy.rmse(predictions), 4)